In [1]:
# export
import Financial_Indicators as fi
import numpy as np
import pandas as pd
import pickle

In [2]:
# export
def load_data():
    file = open("df_all.pkl","rb")
    df_all = pickle.load(file)
    return df_all

In [23]:
# export
def f_temp(x, thr):
    if x >= thr:
        return 1
    elif x <= 1- thr:
        return -1
    else:
        return 0


def f_temp3(x, thr):
    idx = np.argmax(x)
    if (idx == 0) & (x[idx] >= thr):
        return -1
    elif (idx == 1) & (x[idx] >= thr):
        return 0
    elif (idx == 2) & (x[idx] >= thr):
        return 1
    else:
        return -99    
    
    
def f_classif_report(clf, X, y, thr = 0.5):
    orig_len = len(y)
    
    if len(clf.predict_proba(X)[0]) == 3:
        y_pred = [f_temp3(x, thr) for x in clf.predict_proba(X)]
        np_array = np.array(y_pred)
        item_index = np.where((np_array==-1) | (np_array==1) | (np_array==0) )
        item_index = list(item_index[0])
        y_pred = [y_pred[x] for x  in item_index ]
        y_list = [x[0] for x in  y.values]
        y = [y_list[x] for x  in item_index ]
    elif len(clf.predict_proba(X)[0]) == 2:
        y_pred = [f_temp(x[1], thr) for x in clf.predict_proba(X)]
        np_array = np.array(y_pred)
        item_index = np.where((np_array==-1) | (np_array==1) )
        item_index = list(item_index[0])
        y_pred = [y_pred[x] for x  in item_index ]
        y_list = [x[0] for x in  y.values]
        y = [y_list[x] for x  in item_index ]
    
    last_len = len(y)
    
    loss = 1 - (last_len/orig_len)
    print("Lost on dataset: {x}".format(x = loss ) )  
    print(classification_report(y, y_pred))
    print(confusion_matrix(y, y_pred))
    return y, y_pred, loss


def f_prep_df_to_ML(df, data_part = 'all'):
    df_tmp = df.copy()
    
    if data_part == 'train':
        part = int(np.floor(len(df_tmp)*0.85))
        df_tmp = df_tmp.head(part)
    elif data_part == 'test':
        part = int(np.floor(len(df_tmp)*0.15))
        df_tmp = df_tmp.tail(part)
    else:
        pass ## just get all data
    
    df_tmp.dropna(inplace = True)
    y = df_tmp.loc[:, ["Target"]]
    X = df_tmp.loc[:, [ x for x in df_tmp.columns if x not in (["Target" 
                                                                ,"coin" 
                                                                ,"next1price" 
                                                                ,"next2price" 
                                                                ,"next3price" 
                                                                ,"price"
                                                               # ,"is_day_end"
                                                               # ,"last_time"
                                                               ])  ]]

    return X, y 


In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
#from sklearn.svm import SVC

df_all_coins = load_data()





In [31]:
#df_tmp.tail(10.0)
df_tmp = df_all_coins.loc[df_all_coins.coin=='DOGEUSDT', : ]
    
    #df_tmp = df_all_coins.loc[(df_all_coins.coin==coin) & (df_all_coins.Target != 0), : ]
X_train, y_train = f_prep_df_to_ML(df_tmp, data_part = 'test')
X_train

,vol,tradefreq,last_time,is_day_end,rsi14,rsi28,rsi42,rsi48,rsi72,macd12,signal12,hist12,macd6,signal6,hist6,macd12_ratio,signal12_ratio
dt,,,,,,,,,,,,,,,,,
2021-03-27 06:00:00,59912208.0,7260.0,0,0,55.352585,49.914839,48.233275,47.986461,48.004938,0.000276,0.000071,0.000205,0.000343,0.000362,-0.000018,0.005137,0.001328
2021-03-27 08:00:00,285865236.0,24061.0,0,0,65.787129,55.884666,52.412682,51.667310,50.413317,0.000404,0.000138,0.000266,0.000513,0.000422,0.000091,0.007283,0.002486
2021-03-27 10:00:00,170795805.0,16344.0,0,0,55.689561,51.315116,49.465462,49.110321,48.763217,0.000408,0.000192,0.000216,0.000431,0.000426,0.000006,0.007523,0.003539
2021-03-27 12:00:00,109295025.0,13213.0,0,0,57.411903,52.304630,50.167762,49.731932,49.175959,0.000428,0.000239,0.000189,0.000402,0.000416,-0.000014,0.007854,0.004386
2021-03-27 14:00:00,60799283.0,8727.0,0,0,57.444054,52.322767,50.180562,49.743247,49.183452,0.000439,0.000279,0.000160,0.000368,0.000397,-0.000029,0.008053,0.005119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-21 20:00:00,731352529.0,288239.0,0,0,40.205937,41.752713,42.953556,43.439690,45.097833,-0.011965,-0.009039,-0.002926,-0.010827,-0.008628,-0.002199,-0.033854,-0.025574
2021-05-21 22:00:00,332106093.3,158304.0,0,0,41.651698,42.430761,43.386582,43.811855,45.332700,-0.012122,-0.009655,-0.002467,-0.009723,-0.009066,-0.000657,-0.033882,-0.026987
2021-05-22 00:00:00,227467811.4,86450.0,0,1,40.533174,41.894779,43.031970,43.502050,45.125238,-0.012436,-0.010211,-0.002224,-0.009237,-0.009134,-0.000103,-0.035212,-0.028914


In [32]:
model_dict = {}
list_all = []
#for coin in df_all_coins.coin.unique():
for coin in ['BTCUSDT']: #DOGEUSDT, LUNAUSDT, XRPUSDT,UNIUSDT, FILUSDT
    df_tmp = df_all_coins.loc[df_all_coins.coin==coin, : ]
    
    #df_tmp = df_all_coins.loc[(df_all_coins.coin==coin) & (df_all_coins.Target != 0), : ]
    X_train, y_train = f_prep_df_to_ML(df_tmp, data_part = 'train')
    X_test, y_test = f_prep_df_to_ML(df_tmp, data_part = 'test')
    
    clf = RandomForestClassifier(n_estimators=10 ,random_state=0, max_depth = 5, class_weight = "balanced")
    #clf.fit(X , y)
    clf = OneVsRestClassifier(clf).fit(X_train, y_train)

    del df_tmp
    
    thr = 0.4
    is_model_valid = False
    while thr < 0.91:
        try:
            y_true, y_pr, loss_train = f_classif_report(clf, X_train, y_train, thr)
            f1_train = f1_score(y_true, y_pr, average='macro')
            print("TRAIN: {coin} model trial, f1_Score: {f1_Score} and loss is: {loss}".format(coin =coin ,f1_Score = str(f1_train), loss = str(loss_train) ))
            
            y_true, y_pr, loss_test = f_classif_report(clf, X_test, y_test, thr)
            f1_test = f1_score(y_true, y_pr, average='macro')
            print("TEST: {coin} model trial, f1_Score: {f1_Score} and loss is: {loss}".format(coin =coin ,f1_Score = str(f1_test), loss = str(loss_test) ))
            
            if (f1_train > 0.75) & (f1_test > 0.75) & (loss_train <= 0.7) & (loss_test <= 0.7):
                is_model_valid = True
                print("model is OK")
                break;
            #thr = thr + 0.02       
        except Exception as e:
            print(str(e))
            pass
        thr = thr + 0.02
    
    if is_model_valid:
        model_dict.update ({coin: [clf, thr]})
    #graph for feature importance

    #from sklearn.metrics import f1_score
    #f1_score(y, y_pred, average='macro')    
    print('**********************')

pickle_out = open("model_dict.pkl","wb")
pickle.dump(model_dict, pickle_out)
pickle_out.close()    

Lost on dataset: 0.11455847255369933
              precision    recall  f1-score   support

          -1       0.39      0.63      0.48       267
           0       0.92      0.78      0.84      2620
           1       0.42      0.65      0.51       452

    accuracy                           0.75      3339
   macro avg       0.58      0.68      0.61      3339
weighted avg       0.81      0.75      0.77      3339

[[ 167   67   33]
 [ 215 2034  371]
 [  48  109  295]]
TRAIN: BTCUSDT model trial, f1_Score: 0.6113434408702899 and loss is: 0.11455847255369933
Lost on dataset: 0.14328358208955227
              precision    recall  f1-score   support

          -1       0.19      0.27      0.22        95
           0       0.74      0.06      0.11       404
           1       0.15      0.79      0.25        75

    accuracy                           0.19       574
   macro avg       0.36      0.37      0.19       574
weighted avg       0.57      0.19      0.14       574

[[ 26   1  68]
 [10

Lost on dataset: 0.8865671641791044
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        20
           0       0.00      0.00      0.00        37
           1       0.25      1.00      0.40        19

    accuracy                           0.25        76
   macro avg       0.08      0.33      0.13        76
weighted avg       0.06      0.25      0.10        76

[[ 0  0 20]
 [ 0  0 37]
 [ 0  0 19]]
TEST: BTCUSDT model trial, f1_Score: 0.13333333333333333 and loss is: 0.8865671641791044
Lost on dataset: 0.6449217714134181
              precision    recall  f1-score   support

          -1       1.00      0.15      0.26        20
           0       0.96      1.00      0.98      1265
           1       0.80      0.44      0.57        54

    accuracy                           0.96      1339
   macro avg       0.92      0.53      0.60      1339
weighted avg       0.96      0.96      0.95      1339

[[   3   17    0]
 [   0 1259    6]
 [  

Lost on dataset: 1.0
max() arg is an empty sequence
Lost on dataset: 0.9085123309466985
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       344
           1       0.00      0.00      0.00         1

    accuracy                           1.00       345
   macro avg       0.50      0.50      0.50       345
weighted avg       0.99      1.00      1.00       345

[[344   0]
 [  1   0]]
TRAIN: BTCUSDT model trial, f1_Score: 0.4992743105950653 and loss is: 0.9085123309466985
Lost on dataset: 1.0
max() arg is an empty sequence
Lost on dataset: 0.9342349509413949
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       247
           1       0.00      0.00      0.00         1

    accuracy                           1.00       248
   macro avg       0.50      0.50      0.50       248
weighted avg       0.99      1.00      0.99       248

[[247   0]
 [  1   0]]
TRAIN: BTCUSDT model trial, f1_Scor

In [77]:
X_c = X.copy()

X.loc[:,"proba_1"] = [x[0] for x in clf.predict_proba(X_c)] 
X.loc[:,"proba_0"] = [x[1] for x in clf.predict_proba(X_c)] 
X.loc[:,"proba1"] = [x[2] for x in clf.predict_proba(X_c)] 
X


,vol,tradefreq,last_time,is_day_end,rsi14,rsi28,rsi42,rsi48,rsi72,macd12,signal12,hist12,macd6,signal6,hist6,macd12_ratio,signal12_ratio,proba_1,proba_0,proba1
dt,,,,,,,,,,,,,,,,,,,,
2020-10-27 16:00:00,45873.6100,6411.0,0,0,55.254615,48.234309,43.667941,42.375068,40.993625,1.510030,1.598980,-0.088950,0.409555,0.692531,-0.282976,0.043367,0.045922,0.601366,0.278533,0.120101
2020-10-27 20:00:00,78667.0400,10851.0,0,0,56.061620,48.628672,43.930381,42.606430,41.147945,1.440128,1.567209,-0.127081,0.383433,0.568892,-0.185459,0.040948,0.044562,0.642128,0.276630,0.081242
2020-10-28 00:00:00,96548.6000,14263.0,0,1,54.049344,47.903238,43.532150,42.274166,40.939819,1.318459,1.517459,-0.199000,0.262347,0.446274,-0.183927,0.038218,0.043986,0.629541,0.290809,0.079650
2020-10-28 04:00:00,86359.9200,10470.0,0,0,51.957380,47.143573,43.115122,41.926358,40.722255,1.155928,1.445153,-0.289225,0.085508,0.301968,-0.216459,0.034199,0.042756,0.540733,0.309946,0.149321
2020-10-28 08:00:00,78326.5500,8598.0,0,0,49.069466,46.073706,42.525874,41.434776,40.414732,0.942336,1.344590,-0.402254,-0.155205,0.119098,-0.274304,0.028718,0.040977,0.451574,0.438367,0.110058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-07 20:00:00,33687.0712,11650.0,0,0,42.987091,46.628092,47.881582,48.208699,49.257931,-0.614128,-0.502423,-0.111704,-0.532539,-0.297935,-0.234604,-0.004089,-0.003345,0.164687,0.592489,0.242823
2021-05-07 22:00:00,74622.9410,22144.0,0,0,43.337387,46.766184,47.962521,48.276400,49.297297,-0.718776,-0.545694,-0.173082,-0.654988,-0.440756,-0.214232,-0.004781,-0.003630,0.236406,0.481662,0.281932
2021-05-08 00:00:00,78007.7876,17344.0,0,1,46.599010,48.052808,48.715625,48.906039,49.662851,-0.702778,-0.577111,-0.125667,-0.541964,-0.481239,-0.060725,-0.004638,-0.003809,0.141892,0.633876,0.224232


In [75]:
model_dict.get("DOGEUSDT")

[OneVsRestClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                      max_depth=5,
                                                      n_estimators=10,
                                                      random_state=0)),
 0.4800000000000001]

In [25]:
model_dict

{'DOGEUSDT': [OneVsRestClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                       max_depth=5,
                                                       n_estimators=10,
                                                       random_state=0)),
  0.4600000000000001]}

In [76]:
from exporter import export
export("MLTrain.ipynb","MLTrain.py")